In [38]:
import joblib
import pandas as pd
import numpy as np

In [39]:
# --- Load Data ---
df_demog = pd.read_excel('Data/DataScientist_CaseStudy_Dataset.xlsx', sheet_name='Soc_Dem')
df_bal = pd.read_excel('Data/DataScientist_CaseStudy_Dataset.xlsx', sheet_name='Products_ActBalance')
df_in_out = pd.read_excel('Data/DataScientist_CaseStudy_Dataset.xlsx', sheet_name='Inflow_Outflow')
df_target = pd.read_excel('Data/DataScientist_CaseStudy_Dataset.xlsx', sheet_name='Sales_Revenues')
df_desc = pd.read_excel('Data/DataScientist_CaseStudy_Dataset.xlsx', sheet_name='Description')

# Data merge pipeline
df_master = (df_demog
             .merge(df_in_out, on='Client')
             .merge(df_bal, on='Client'))

df_master_market = df_master[~df_master['Client'].isin(client for client in df_target['Client'])].fillna(0)

# Adding feature engineering
df_master_market['Avg_CA_bal'] = df_master_market['ActBal_CA']/df_master_market['Count_CA']
df_master_market['Avg_SA_bal'] = df_master_market['ActBal_SA']/df_master_market['Count_SA']
df_master_market['Avg_MF_bal'] = df_master_market['ActBal_MF']/df_master_market['Count_MF']
df_master_market['utilization_CA'] = np.minimum(100,df_master_market['VolumeDeb_CA']/df_master_market['ActBal_CA'])

df_master_market = df_master_market.drop(columns=['Client'])

In [41]:
df_market_final = df_master_market.copy()
products = ["CL", "CC", "MF"]

for product in products:

    # Predict probabilities
    model = joblib.load(f"Model/Sale_{product}_model.joblib")
    binning_process = joblib.load(f"Model/Sale_{product}_binning_process.joblib")
    selected_features = joblib.load(f"Model/Sale_{product}_selected_features.joblib")

    # --- Load or Prepare New Data for Scoring ---
    X_new_woe = binning_process.transform(df_market_final, metric="woe")
    X_new_final = X_new_woe[selected_features]
    pred_proba = model.predict_proba(X_new_final)[:, 1]

    df_market_final[f"prob_{product}"] = pred_proba

    # Predict Revenue
    # Load the model from file

    df_master_market['Sex'] = df_master_market['Sex'].astype('category')
    regressor_loaded = joblib.load(f"Model/Revenue_{product}_model.joblib")
    # Predict
    y_pred = regressor_loaded.predict(df_master_market)
    df_market_final[f"rev_{product}"] = y_pred

    df_market_final[f"expected_revenue_{product}"] = df_market_final[f"prob_{product}"] * df_market_final[f"rev_{product}"]


C:\Users\joeytian\anaconda3\Lib\site-packages\optbinning\binning\transformations.py:38: RuntimeWarning: invalid value encountered in log
  return np.log((1. / event_rate - 1) * n_event / n_nonevent)
C:\Users\joeytian\anaconda3\Lib\site-packages\optbinning\binning\transformations.py:38: RuntimeWarning: invalid value encountered in log
  return np.log((1. / event_rate - 1) * n_event / n_nonevent)
C:\Users\joeytian\anaconda3\Lib\site-packages\optbinning\binning\transformations.py:38: RuntimeWarning: divide by zero encountered in log
  return np.log((1. / event_rate - 1) * n_event / n_nonevent)


In [42]:
# Select the expected revenue columns
expected_revenue_cols = [f"expected_revenue_{p}" for p in products]

# Max expected revenue across products
df_market_final["expected_revenue"] = df_market_final[expected_revenue_cols].max(axis=1)

# Product with highest expected revenue (argmax)
df_market_final["selected_product"] = df_market_final[expected_revenue_cols].idxmax(axis=1).str.replace("expected_revenue_", "")

In [43]:
df_top100 = df_market_final.sort_values(by="expected_revenue", ascending=False).head(100)
df_top100.to_csv("Client_List_Final.csv")

In [45]:
df_top100['expected_revenue'].sum()

840.8758145908482